In [1]:
import urllib
from bs4 import BeautifulSoup
import re

In [9]:
def main():
    baseurl='https://movie.douban.com/top250?start='
    

    
    
    datalist = getData(baseurl)
    savepath = ".\\豆瓣电影Top250.xls"
    saveData(savepath)

In [12]:
    #定义正则表达式对象，方便在getData()中用正则表达式查找，加r是防止程序误以为有转义字符
    #电影链接：
    findLink = re.compile(r'<a href="(.*?)">') 
    #图片链接：
    findImgSrc = re.compile(r'<img.*src="(.*?)">',re.S)#re.S是忽略换行符，因为链接里面可能出现奇怪字符
    #电影名字：
    findTitle = re.compile(r'<span class="title">(.*)</span>')
    #影片评分：
    findRating = re.compile(r'<span class="rating_num" property="v:average">(.*)</span>')
    #评价人数：
    findJudge = re.compile(r'<span>(\d*)人评价</span>')
    #概况
    findInq = re.compile(r'<span class="inq">(.*)</span>')
    #影片相关内容
    findBd = re.compile(r'<p class="">(.*?)</p>',re.S)

In [14]:
#爬取网页
def getData(baseurl):
    datalist = []
    for i in range(0,1):   #调用获取页面信息的函数，10次
        url = baseurl + str(i*25)
        html = askURL(url)   #保存获取到的网页源码
        
        #逐一解析数据
        soup = BeautifulSoup(html,"html.parser")#用漂亮汤的html.parser解析网页，放进soup里
        for item in soup.find_all('div',class_="item"): #查找一部电影
            data = []  #保存一部电影的所有信息
            item = str(item) #转成字符串，方便用正则表达式
            
            link = re.findall(findLink,item)[0] #找到电影对应的链接
            data.append(link)#添加链接
            
            imgSrc = re.findall(findImgSrc,item)[0] #找图片
            data.append(imgSrc)
            
            titles = re.findall(findTitle,item)#找title
            if(len(titles)==2):
                ctitle = titles[0]#中文title
                data.append(ctitle)
                otitle = titles[1].replace("/","")#外文title,顺便去掉无关符号/
                data.append(otitle)
            else:
                data.append(titles[0])
                data.append(' ') #如果没有外文名，也要用空格占个位置 
                
            rating = re.findall(findRating,item)[0]#找评分
            data.append(rating)
            
            judgeNum = re.findall(findJudge,item)[0]#找评价人数
            data.append(judgeNum)
            
            inq = re.findall(findInq,item)#找概述,概述可能没有
            if len(inq)!=0:
                inq = inq[0].replace("。","") 
                data.append(inq)
            else:
                data.append(" ")
                
            bd = re.findall(findBd,item)[0]#找影片相关内容
            bd = re.sub('<br(/s+)?/>(\s+)?'," ",bd) #去掉<br/>
            bd = re.sub('/'," ",bd) #替换/
            data.append(bd.strip()) #strip去掉空格
           
            #将处理好的一部电影信息再作为一个元素放入datalist
            datalist.append(data)

    return datalist

In [15]:
#测试getData('https://movie.douban.com/top250?start=')

[['https://movie.douban.com/subject/1292052/', 'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p480747492.jpg" width="100"/>\n</a>\n</div>\n<div class="info', '肖申克的救赎', '\xa0\xa0The Shawshank Redemption', '9.7', '2247978', '希望让人自由', '导演: 弗兰克·德拉邦特 Frank Darabont\xa0\xa0\xa0主演: 蒂姆·罗宾斯 Tim Robbins  ... 1994\xa0 \xa0美国\xa0 \xa0犯罪 剧情'], ['https://movie.douban.com/subject/1291546/', 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2561716440.jpg" width="100"/>\n</a>\n</div>\n<div class="info', '霸王别姬', ' ', '9.6', '1669221', '风华绝代', '导演: 陈凯歌 Kaige Chen\xa0\xa0\xa0主演: 张国荣 Leslie Cheung   张丰毅 Fengyi Zha... 1993\xa0 \xa0中国大陆 中国香港\xa0 \xa0剧情 爱情 同性'], ['https://movie.douban.com/subject/1292720/', 'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p2372307693.jpg" width="100"/>\n</a>\n</div>\n<div class="info', '阿甘正传', '\xa0\xa0Forrest Gump', '9.5', '1692907', '一部美国近现代史', '导演: 罗伯特·泽米吉斯 Robert Zemeckis\xa0\xa0\xa0主演: 汤姆·汉克斯 Tom Hanks   ... 1994\xa0 \xa0美国\xa0 \xa0剧情 

[['https://movie.douban.com/subject/1292052/',
  'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p480747492.jpg" width="100"/>\n</a>\n</div>\n<div class="info',
  '肖申克的救赎',
  '\xa0\xa0The Shawshank Redemption',
  '9.7',
  '2247978',
  '希望让人自由',
  '导演: 弗兰克·德拉邦特 Frank Darabont\xa0\xa0\xa0主演: 蒂姆·罗宾斯 Tim Robbins  ... 1994\xa0 \xa0美国\xa0 \xa0犯罪 剧情'],
 ['https://movie.douban.com/subject/1291546/',
  'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2561716440.jpg" width="100"/>\n</a>\n</div>\n<div class="info',
  '霸王别姬',
  ' ',
  '9.6',
  '1669221',
  '风华绝代',
  '导演: 陈凯歌 Kaige Chen\xa0\xa0\xa0主演: 张国荣 Leslie Cheung   张丰毅 Fengyi Zha... 1993\xa0 \xa0中国大陆 中国香港\xa0 \xa0剧情 爱情 同性'],
 ['https://movie.douban.com/subject/1292720/',
  'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p2372307693.jpg" width="100"/>\n</a>\n</div>\n<div class="info',
  '阿甘正传',
  '\xa0\xa0Forrest Gump',
  '9.5',
  '1692907',
  '一部美国近现代史',
  '导演: 罗伯特·泽米吉斯 Robert Zemeckis\xa0\xa0\xa0主演: 汤姆·汉克斯

In [5]:

def askURL(url):
    head = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36"}
    #以上是我的chrome浏览器的头部信息
    request = urllib.request.Request(url,headers=head)
    html = ""
    try:
        response = urllib.request.urlopen(request)
        html = response.read().decode("utf-8")
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
            print(e.code)
        if hasattr(e,"reason"):
            print(e.reason)
    return html

In [ ]:
#测试askURL('https://movie.douban.com/top250?start=0&filter=')